# Prédictions avec scikitlearn

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup as BS
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time
from serde import serialize, deserialize
from serde.json import to_json, from_json
import pandas as pd
from requests import get 
from dataclasses import dataclass
from pays import Countries

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [5]:
df = pd.read_pickle("my_df.pkl")

In [6]:
df

,description,prix,lieux,duree,diff,theme
0,plaisirs neige &#224; sixt-fer-&#224;-cheval,690,metropole,8,Initié,multi-activités
1,haute clar&#233;e &quot;espace trappeur&quot;,860,metropole,7,Découverte,multi-activités
2,aventures hivernales sur le plateau de l&#39;a...,720,metropole,6,Découverte,multi-activités
3,neige et soleil du queyras en famille,730,metropole,7,Découverte,multi-activités
4,plaisirs neige dans les aravis,830,metropole,7,Découverte,multi-activités
...,...,...,...,...,...,...
383,plongée à la réunion,1540,dest_éloignée,10,Initié,multi-activités
384,randonnée dans le parc naturel de topes de col...,1690,dest_éloignée,14,Découverte,rando nature
385,"randonnée jusqu'au mont ryten, vue sur kvalvika",1930,dest_éloignée,15,Découverte,rando nature
386,drapée masai flottant dans les terres du rift,2350,dest_éloignée,8,Découverte,rando nature


In [9]:
df.shape

(388, 6)

In [17]:
X = df.drop(["description", "prix"], axis = 1)
y = df["prix"]

In [19]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y)

In [21]:
results = dict()